# **과제2 : 챗봇 모델 완성하기**

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [ ]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [62]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [ ]:
data = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
data.head()

* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./database



In [2]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
database = Chroma(persist_directory="./database", embedding_function = embeddings )

* 데이터 입력
    * 이미 입력된 데이터는 제외하고 나머지 입력하기

In [ ]:
# 각 행의 데이터를 Document 객체로 변환 
documents = [Document(page_content=text) for text in data['QA'].tolist()]

# 데이터프레임에서 문서 추가
database.add_documents(documents)

* 입력된 데이터 조회

In [ ]:
database.get()

## **3.RAG+LLM모델**

* 모델 선언 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 5개 가져오도록 설정
    * memory
* 결과
    * 유사도 높은 문장 3개와 유사도 점수도 함께 출력하도록 코드 작성
    * 질문 history 관리를 위한 이력 저장
        * timestamp
        * 질문
        * 유사도 3개
        * 답변
        * 그외


In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer", 
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, 
                                           return_source_documents=True,  output_key="answer")

* 모델 사용하기
    * 에이블스쿨 지원 자격 관련 질문을 하고 답을 확인해 봅시다.

In [ ]:
query = '지원하는데 나이 제한이 있나요?'
result = qa(query)

print(result["answer"])